In [ ]:
!pip install dnspython
!pip install ipynb

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
# Import all functions from Funtions.ipynb
from ipynb.fs.full.Functions import *

#PostgreSQL access data
host = "bda_gr4_database"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("domain_analysis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", "domain") \
                .option("user", user) \
                .option("password", password) \
                .load()

In [ ]:
# delete
# Get SOA information error from domain  
def getSOAInformation_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'SOA')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.exception.Timeout:
        return 4

In [ ]:
udf_getIPv6Records = udf(getIPv6Records, StringType())
udf_getSOAInformation = udf(getSOAInformation, ArrayType(StringType()))
udf_getSOAInformation_error = udf(getSOAInformation_error, ArrayType(StringType()))
udf_getNameServers = udf(getNameServers, ArrayType(StringType()))
udf_getNameServers_error = udf(getNameServers_error, IntegerType())

In [ ]:
domains_df = domains_df.withColumn("aaaa_record", udf_getIPv6Records("top_level_domain"))
domains_df = domains_df.withColumn("soa_information", udf_getSOAInformation("top_level_domain"))
domains_df = domains_df.withColumn("soa_information_error", udf_getSOAInformation_error("top_level_domain"))
domains_df = domains_df.withColumn("nameservers", udf_getNameServers("top_level_domain"))
domains_df = domains_df.withColumn("nameservers_error", udf_getNameServers_error("top_level_domain"))

In [ ]:
# Change ArrayType<String> into String as preparation for information separation
domains_df = domains_df.withColumn("soa_infos_rep", concat_ws(" ", "soa_information"))

# Split SOA information into separate columns (all String)
split_col = split(domains_df['soa_infos_rep'], ' ')
domains_df = domains_df.withColumn('mname', split_col.getItem(0))
domains_df = domains_df.withColumn('refresh', split_col.getItem(3))
domains_df = domains_df.withColumn('minimum', split_col.getItem(6))

#
def replace_empty_strings(x):
    return when(col(x) == "", None).otherwise(col(x))

domains_df = domains_df.withColumn("mname", replace_empty_strings("mname"))

In [ ]:
# Remove last dot per soa mname
domains_df = domains_df.withColumn('mname', regexp_replace('mname', '.$', ''))   

# Remove last dot per nameserver entry
lambda_dot_remove = lambda arr: [x[:-1] for x in arr]
def fn_remove_dot(arr): return None if arr == None else lambda_dot_remove(arr)
udf_remove_last_char_in_array = udf(fn_remove_dot, ArrayType(StringType()))

domains_df = domains_df \
    .select("*") \
    .withColumn('nameservers', udf_remove_last_char_in_array(col('nameservers')))

In [ ]:
# change dtypes of seconds to int
domains_df = domains_df.withColumn("refresh", domains_df["refresh"].cast(IntegerType()))
domains_df = domains_df.withColumn("minimum", domains_df["minimum"].cast(IntegerType()))

#domains_df.select('mname').show(20, False)
#domains_df.show(5)

In [ ]:
# Count the values of each column with descending sort and show it
# domains_df.groupBy("Top-Level-Domain").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("A-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("MX-Record_Checked").count().sort(desc("count")).show(20, False)
# domains_df.groupBy("Redirection").count().sort(desc("count")).show(20, False)

In [ ]:
# Count the occrence of SOA records
soa_mname_count = domains_df.withColumn('mname', (col('mname'))) \
        .groupBy('mname') \
        .count()

soa_mname_count_top_ten_df = soa_mname_count.orderBy(['count'], ascending = [False]).limit(10)
soa_mname_count_top_ten_df.show()

In [ ]:
# domains_df = domains_df.withColumn('mx_record_count', explode(col('mx_record'))) \
#         .groupBy('mx_record_count') \
#         .count() \
#         .sort(desc("count"))

domains_df = domains_df.withColumn('mx_record_checked_count', explode(col('mx_record_checked'))) \
        .groupBy('mx_record_checked_count') \
        .count() \
        .sort(desc("count"))

# domains_df = domains_df.withColumn('a_record_count', explode(col('a_record'))) \
#         .groupBy('a_record_count') \
#         .count() \
#         .sort(desc("count"))

domains_df = domains_df.withColumn('a_record_checked', explode(col('a_record_checked'))) \
        .groupBy('a_record_checked') \
        .count() \
        .sort(desc("count"))

domains_df = domains_df.withColumn('m_name', explode(col('m_name'))) \
        .groupBy('m_name') \
        .count() \
        .sort(desc("count"))

domains_df = domains_df.withColumn('nameservers_count', explode(col('nameservers'))) \
        .groupBy('nameservers_count') \
        .count() \
        .sort(desc("count"))

domains_df = domains_df.drop('mx_record').drop('a_record').drop('soa_information').drop('soa_infos_rep')

In [ ]:
domains_df.show(5)

In [ ]:
# Write the data frame to the PostgreSQL database
domains_df.repartition(8).write \
    .format("jdbc") \
    .option("url", connection) \
    .option("dbtable", "domain_enhanced") \
    .option("user", user) \
    .option("batchsize", 10000) \
    .option("password", password) \
    .mode("append") \
    .save()